# Pipelime Command Line Interface

The pipelime command line interface is a powerful tool to automate data processing.
First, you can get help simply typing `pipelime`, `pipelime help`, `pipelime --help` or
even `pipelime -h`:

In [ ]:
!pipelime

The CLI is built around the concept of `Pipelime Command`, which encapsulates an
operation and makes it available both to the CLI and usual python scripting. Such
commands are dynamically loaded at runtime, so you can always run a third-party command
just by setting its full class path, eg, `my_package.my_module.MyCommand` or
`path/to/my_module.py:MyCommand`. Alternatively, let pipelime find and load your command
by setting `--module my_package.my_module` or `--module path/to/my_module.py`, then
refer to it by its pydantic title (see **#TODO REF**).

The list of available commands and sequence operators (more on this later in this doc)
can be retrieved with:

In [ ]:
!pipelime list

Now, to get help for a specific command or sequence operator, just type
`pipelime help <cmd>`, `pipelime <cmd> help`, `pipelime --help <cmd>`, etc,
eg (best viewed in a *real* terminal window):

In [ ]:
!pipelime --help clone

Also, the same help can be printed during an interactive session by explicitly calling the printer: 

In [ ]:
from pipelime.cli.pretty_print import print_model_info
from pipelime.commands import CloneCommand

print_model_info(CloneCommand)

## Running A Command

As you can see above, the *clone* command:
* needs 3 arguments: **input** (required), **output** (required) and **grabber** (optional)
* each argument is, in fact, an *interface* encapsulating a full range of options in a tree-like structure

When you call *clone* through the pipelime cli, you can set all those options in different ways, ie:
* pydash-like key paths prefixed with "+", where the "." separates nested keys and "[]"
indexes a list, eg, `+input.folder path/to/folder`.
* a json/yaml configuration file specified as `--config path/to/cfg.yaml`. Note that command line
options update and override config file definitions.

In [ ]:
!pipelime clone +input.folder ../../tests/sample_data/datasets/underfolder_minimnist +output.folder ./clone_out +output.exists_ok=True

Likewise, the `CloneCommand` can be created and run in a python script as well:

In [ ]:
from pipelime.commands import CloneCommand
from pipelime.cli.pretty_print import print_command_outputs

cmd = CloneCommand(
    input={"folder": "../../tests/sample_data/datasets/underfolder_minimnist"},
    output={"folder": "./clone_out", "exists_ok": True},
)
cmd()
print_command_outputs(cmd)

## Executing A Graph Of Commands

Multiple commands can be chained ad executed as a Direct Acyclic Graph (DAG) by the *run* command (`RunCommand`):

In [ ]:
from pipelime.commands import RunCommand
print_model_info(RunCommand)

The `nodes` attribute is a mapping of nodes, where the keys are the nodes' names and the values the actual commands to execute. As a practical example, look at the *complex_dag.yaml* file. It may seem intimidating at first, but we can easily understand the data flow by drawing it:

In [ ]:
!pipelime draw --config complex_dag.yaml

Oops! Something went wrong...
As the error message says, we need to specify some variables. To get a full list,
just audit the configuration file:

In [ ]:
!pipelime audit --config complex_dag.yaml

The `complex_params.yaml` file defines such variables, except for `params.root_folder`, which might be OS-dependent and so it must be defined on the command line using the special `!` prefix:

In [ ]:
!pipelime audit --config complex_dag.yaml --context complex_params.yaml !params.root_folder=./output

Now we are ready to inspect and run the computation graph (NB: manually remove the `output` folder before each run):

In [ ]:
!pipelime draw --config complex_dag.yaml --context complex_params.yaml !params.root_folder=./output

In [ ]:
!pipelime run --config complex_dag.yaml --context complex_params.yaml !params.root_folder=./output